# Assignment 5 - Data Crawling from Website

이번 과제에서 학생들은 웹사이트에서 데이터를 수집하고 원하는 형태로 정리하는 연습을 하게될 것이다.

* 제출 방법: ETL (11/20 자정까지)
* 파일 이름: A5-학번-이름(영어로).iphynb 형식으로 (예: A5-13403-999-jiyoon.iphynb)

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import random
import re
import json

rand_url = "http://www.todayhumor.co.kr/board/view.php?table=bestofbest&no=287215&s_no=287215&page=1"
#rand_url = "http://www.todayhumor.co.kr/board/view.php?table=bestofbest&no=287929&s_no=287929&page=1"
rand_url = "http://www.todayhumor.co.kr/board/view.php?table=bestofbest&no=287974&s_no=287974&page=1"
    
def getInfos(url):
    
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = urllib.request.Request(url,headers=hdr)

    doc = ""
    with urllib.request.urlopen(req) as url:
        doc = url.read()

    soup = BeautifulSoup(doc, "html.parser")

    ## get Writer Infomation
    writerInfoContents = soup.find_all("div", class_="writerInfoContents")
    divTags = writerInfoContents[0].find_all("div")

    contentID = divTags[0].text.split(' : ')[1]
    writerName = (divTags[1].find_all("b"))[0].text   # 글쓴이 닉네임
    writerInfo_ = (divTags[1].find_all("span"))[-1].text
    writerSignInDate = re.split(':| |\)',writerInfo_)[1]   # 글쓴이 가입 날짜
    writerVisitingCount = re.split(':| |\)',writerInfo_)[3]  # 방문 횟수
    recommendCount = divTags[2].find_all("span")[0].text   # 추천수
    viewCount = divTags[3].text.split(' : ')[1]            # 조회수 
    commentCount = re.split(' : |개',divTags[5].text)[1]   # 댓글 수 
    BOBTime = divTags[6].text.split(' : ')[1]              # 베스트 등록 시간
    postTIme = divTags[7].text.split(' : ')[1]             # 게시 시간

    #### 게시자의 다른 글 개수 ####
    baseURL = "http://www.todayhumor.co.kr/board/"
    writerProfileURLSuffix = divTags[1].find_all("a")[0]["href"]
    writerProfileURL = baseURL + writerProfileURLSuffix
    writerProfileReq = urllib.request.Request(writerProfileURL,headers=hdr)

    writerProfileDoc = ""
    with urllib.request.urlopen(writerProfileReq) as url:
        writerProfileDoc = url.read()

    writerProfileSoup = BeautifulSoup(writerProfileDoc, "html.parser")    
    table_list = writerProfileSoup.find_all("table", class_="table_list")
    normalPostCount = table_list[0].find("a").text         # 유저가 쓴 글 개수

    baseURL = "http://www.todayhumor.co.kr"
    member_menu_box = writerProfileSoup.find_all("div", class_="member_menu_box")
    writerProfileURLs = member_menu_box[0].find_all("a")
    writerProfileBestURL = baseURL + writerProfileURLs[1]["href"]
    writerProfileBOBURL = baseURL + writerProfileURLs[2]["href"]

    writerProfileReq = urllib.request.Request(writerProfileBestURL,headers=hdr)
    writerProfileDoc = ""
    with urllib.request.urlopen(writerProfileReq) as url:
        writerProfileDoc = url.read()

    writerProfileSoup = BeautifulSoup(writerProfileDoc, "html.parser")    
    table_list = writerProfileSoup.find_all("table", class_="table_list")
    bestPostCount = table_list[0].find("a").text            # 유저가 쓴 베스트 개수

    writerProfileReq = urllib.request.Request(writerProfileBOBURL,headers=hdr)
    writerProfileDoc = ""
    with urllib.request.urlopen(writerProfileReq) as url:
        writerProfileDoc = url.read()

    writerProfileSoup = BeautifulSoup(writerProfileDoc, "html.parser")    
    table_list = writerProfileSoup.find_all("table", class_="table_list")
    BOBPostCount = table_list[0].find("a").text             # 유저가 쓴 베오베 개수

    ################################

    ## get Bulletin board Info
    viewSubjectDiv = soup.find_all("div", class_="viewSubjectDiv")

    board_icon = viewSubjectDiv[0].find_all("span")[0]["class"][1]    # 게시판 이름 
    title = viewSubjectDiv[0].find_all("div")[0].text.strip()         # 제목

    ## post option : 창작글, 펌글, 베스트 금지, 베오베 금지, 본인삭제금지, 외부펌 금지
    contentContainer = soup.find_all("div", class_="contentContainer")
    postOption = contentContainer[0].find_all("table")

    changjakOption = False # 창작글
    permOption = False # 펌글
    prohibitBestOption = False # 베스트 금지
    prohibitBOBOption = False # 베오베 금지
    prohibitBoninOption = False # 본인삭제 금지
    prohibitOutsidePermOption = False # 외부펌 금지


    if len(postOption) is 2 :  # 첫 <table>은 글 옵션 두번째 <table>은 출처. 즉 테이블이 1개이면 출처만 있고 옵션이 없음

        options = postOption[0].find_all("li")

        for option in options : 
            optionName = option.find_all("div")[1].text
            if optionName == "창작글" :
                changjakOption = True
            elif optionName == "펌글" :
                permOption = True
            elif optionName == "베스트금지" :
                prohibitBestOption = True
            elif optionName == "베오베금지" :
                prohibitBOBOption = True
            elif optionName == "본인삭제금지" :
                prohibitBoninOption = True
            elif optionName == "외부펌금지" :
                prohibitOutsidePermOption = True    

    ## 본문 내용 가져오기
    # 구해야 할것 :  이미지 개수, 동영상 유무(youtube or 움짤), 텍스트 길이, 키워드

    viewContent = soup.find_all("div", class_="viewContent")
    imgs = viewContent[0].find_all("img") # 본문 삽입된 이미지들
    videos = viewContent[0].find_all("video")
    youtubes = viewContent[0].find_all("iframe")
    texts = viewContent[0].find_all("div")

    textData = []
    for text in texts :
        if (text.text) :
            textData.append(text.text)

    imgCount = len(imgs)   # 이미지 카운트
    videoCount = len(videos)   # 움짤 카운트
    youtubeCount = len(youtubes)   # 유투브(iframe) 개수
    textLineCount = len(textData)   # 텍스트 줄 수


    ## 추천 시간대 
    recommendList = re.split('  | \xa0',soup.find_all("div", id="ok_layer")[0].text)
    recommendDate = []           # 추천 시간대
    i=1
    while i<len(recommendList) : 
        recommendDate.append(recommendList[i])
        i = i + 4

    ## JSON 형태로 변환
    returnJson = {recommendDate[x] : 1 for x in range(len(recommendDate))}
    return json.dumps(returnJson,sort_keys=True,indent=4, separators=(',', ': '))

recommendDate = getInfos(rand_url)

In [2]:
recommendDate

'{\n    "2016/12/07 18:20:20": 1,\n    "2016/12/07 18:23:53": 1,\n    "2016/12/07 18:27:33": 1,\n    "2016/12/07 18:30:41": 1,\n    "2016/12/07 19:07:37": 1,\n    "2016/12/07 19:12:29": 1,\n    "2016/12/07 19:43:56": 1,\n    "2016/12/07 19:49:42": 1,\n    "2016/12/07 20:25:11": 1,\n    "2016/12/07 21:06:55": 1\n}'